## Primerjava playlistov (uporabnikov) in pesmi z vekotrjem atributov

1. Izracun vekotorja - povprecje (mean) ALI sestevek (sum) vseh vrstic 
2. Primerjava 2 vektorjev s funkcijo cosine_similarity

Vsak playlist predstavlja nekega uporabnika, ki poslusa pesmi na playlistu. S primerjavo vektorjev bomo ugotovili podobnost pesmi 2 uporabnikov. 

In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

In [2]:
def playlists(path):
    pls = []
    df = pd.read_csv(path, index_col=0)
    i = 0
    df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    while df_pl.shape[0] > 0:
        pls.append(df_pl)
        i += 1
        df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    
    tab_audio = []
    tab_df = []
    
    for pl in pls:
        df_audio, df_pl = playlist(pl)
        tab_audio.append(df_audio)
        tab_df.append(df_pl)
         
    return [tab_df, tab_audio]

def playlist(df_pl):
    # df_pl.drop_duplicates(subset=['id'], inplace=True)
    # df_pl.set_index("id", inplace=True)

    df_audio = df_pl.loc[:, ['danceability', 'energy', 'loudness', 
                      'speechiness', 'acousticness', 'instrumentalness',
                       'liveness', 'valence', 'tempo', 'duration_ms']]  

    return df_audio, df_pl

Prebermo vse pesmi iz datoteke Playlists5.csv in jih razdelimo v pripadane playliste.
Razdelimo jih s funckijo 'playlists'.

Datoteka vsebuje 24 playlistov razlicnih zanrov.

In [31]:
tab_df_pl = playlists('Data\Playlist\Playlists5.csv')
tab_df = tab_df_pl[0]
audio_df = tab_df_pl[1]
audio_df[2].shape

(137, 10)

In [32]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [33]:
song = audio_df[0].head(1)
playlist1 = audio_df[0]

In [34]:
song

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
id,,,,,,,,,,
17i5jLpzndlQhbS4SrTd0B,0.275,0.157,0.578072,0.0636,0.89,0.842,0.186,0.304,0.343939,0.081501


In [35]:
playlist1['duration_ms'].mean()

0.2777122218694979

In [36]:
def playlist_vector(df_playlist, methode):
    if methode == 'sum':
        playlist_vector = df_playlist.sum(axis = 0)
    elif methode == 'mean':
        playlist_vector = df_playlist.mean(axis = 0)
    return playlist_vector

Iz playlista pesmi izracunamo vektor s funkcijo 'playlist_vector'. 

Vekotr lahko izracunamo iz poprecja vseh vrstic (mean) ali vse vrstice sestejemo skupja (sum).

In [41]:
playlist_vector1_sum = [playlist_vector(playlist1, 'sum')]
playlist_vector1_sum

[danceability         36.977300
 energy               15.234030
 loudness             69.692247
 speechiness           6.565600
 acousticness        135.374000
 instrumentalness    108.083291
 liveness             17.948000
 valence              22.355400
 tempo                71.024276
 duration_ms          40.823697
 dtype: float64]

In [49]:
playlist_vector1_mean = [playlist_vector(playlist1, 'mean')]
playlist_vector1_mean

[danceability        0.251546
 energy              0.103633
 loudness            0.474097
 speechiness         0.044664
 acousticness        0.920912
 instrumentalness    0.735260
 liveness            0.122095
 valence             0.152078
 tempo               0.483158
 duration_ms         0.277712
 dtype: float64]

In [50]:
song_vector = song
song_vector

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
id,,,,,,,,,,
17i5jLpzndlQhbS4SrTd0B,0.275,0.157,0.578072,0.0636,0.89,0.842,0.186,0.304,0.343939,0.081501


In [54]:
print("Podobnost pesmi in playlista: %.2f (mean)" % cosine_similarity(playlist_vector1_mean, song_vector)[0][0])
print("Podobnost pesmi in playlista: %.2f (sum)" % cosine_similarity(playlist_vector1_sum, song_vector)[0][0])

Podobnost pesmi in playlista: 0.97 (mean)
Podobnost pesmi in playlista: 0.97 (sum)


In [55]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Vektorju atributov playlista boma dodali se deleze zanrov pesmi, ki jih ta playlist vsebuje. 

Se pravi bomo vektorju dodali 6 vrednosti, vsaka bo predstavljala delez pesmi tega zanra na playlistu. 

Primer: ce imamo playlist klasike bodo vrednosti 0,0,0,0,0,1 (1 predstavlja 100% delez pesmi klasike).

In [60]:
df_track = pd.read_csv('Data\Genre\Tracks_classification_5.csv', index_col=0)
df_audio = df_track.loc[:, ['danceability', 'energy', 'loudness', 
                      'speechiness', 'acousticness', 'instrumentalness',
                       'liveness', 'valence', 'tempo', 'duration_ms', 'genre']]

In [61]:
df_track['duration_ms'].mean()

0.15357866885277163

In [62]:
print(df_audio.shape)

(81712, 11)


In [63]:
def genre_vector(predict):
    genres = ['pop', 'rock', 'rap', 'edm', 'latin', 'classical']
    genre_v = [0] * 6
    for x in predict:
        index = genres.index(x)
        genre_v[index] += 1
        
    for i, x1 in enumerate(genre_v):
        genre_v[i] = x1/ len(predict)
        
    return genre_v

def make_vector(pl, audio_df):
    playlist1 = audio_df[pl]
    X_test = audio_df[pl] 
    Y_predict = classifier.predict(X_test)
    playlist_vector1 = playlist_vector(playlist1, 'mean')
    
    genre_v = genre_vector(Y_predict)
    return list(playlist_vector1) + genre_v

def make_vector_song(pl, audio_df):
    song = audio_df[pl].sample()
    X_test = audio_df[pl].sample()
    Y_predict = classifier.predict(X_test)
    song_vector1 = playlist_vector(song, 'mean')
    
    genre_v = genre_vector(Y_predict)
    return list(song_vector1) + genre_v

In [64]:
Y = df_audio.pop('genre').values
print(Y)
X = df_audio.values
print(X)

['pop' 'pop' 'pop' ... 'classical' 'classical' 'classical']
[[0.594      0.713      0.87755954 ... 0.328      0.52389744 0.19776463]
 [0.516      0.777      0.88407472 ... 0.408      0.50409981 0.13880682]
 [0.645      0.593      0.85843122 ... 0.299      0.4433766  0.15480744]
 ...
 [0.0961     0.0508     0.55423968 ... 0.0351     0.31729823 0.44921574]
 [0.0723     0.102      0.60457988 ... 0.0369     0.25974764 0.25928797]
 [0.147      0.309      0.74053213 ... 0.335      0.29852858 0.13470945]]


In [65]:
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X, Y) 

RandomForestClassifier()

In [66]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [79]:
# Predict y data with classifier:
X_test = audio_df[0] 

print(X_test.shape)
Y_predict = classifier.predict(X_test)

(147, 10)


In [80]:
Y_predict

array(['classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'clas

In [81]:
print("Delez zanrov v playlistu 14")
genres = ['pop', 'rock', 'rap', 'edm', 'latin', 'classical']
genre_v = genre_vector(Y_predict)
for i, x in enumerate(genre_v):
    print("%s: %.2f" % (genres[i], x))

Delez zanrov v playlistu 14
pop: 0.00
rock: 0.01
rap: 0.00
edm: 0.00
latin: 0.00
classical: 0.99


In [82]:
playlist_vector1_mean

[danceability        0.251546
 energy              0.103633
 loudness            0.474097
 speechiness         0.044664
 acousticness        0.920912
 instrumentalness    0.735260
 liveness            0.122095
 valence             0.152078
 tempo               0.483158
 duration_ms         0.277712
 dtype: float64]

Delez zanrov zdruzimo z vektorjem audio lastnosti s funkcijo 'make_vector'.

In [83]:
make_vector(0, audio_df)

[0.25154625850340134,
 0.1036328571428572,
 0.47409691961369116,
 0.04466394557823131,
 0.9209115646258503,
 0.735260486054422,
 0.12209523809523812,
 0.15207755102040815,
 0.48315834332813323,
 0.2777122218694979,
 0.0,
 0.006802721088435374,
 0.0,
 0.0,
 0.0,
 0.9931972789115646]

In [25]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [89]:
pl1_classic = [make_vector(0, audio_df)]
pl2_classic = [make_vector(2, audio_df)]
pl3_rap = [make_vector(5, audio_df)]
pl4_rap = [make_vector(7, audio_df)]
pl5_latin = [make_vector(18, audio_df)]
pl6_rock = [make_vector(14, audio_df)]
pl7_edm = [make_vector(9, audio_df)]
pl8_pop = [make_vector(21, audio_df)]

In [92]:
print("Podobnost playlistov:\n")
print("Klasika - klasika: %.2f" % cosine_similarity(pl1_classic, pl2_classic)[0][0])
print("Klasika - edm: %.2f" % cosine_similarity(pl1_classic, pl7_edm)[0][0])
print()
print("Rap - rap: %.2f" % cosine_similarity(pl3_rap, pl4_rap)[0][0])
print("Rap - pop: %.2f" % cosine_similarity(pl3_rap, pl8_pop)[0][0])
print("Rap - latin: %.2f" % cosine_similarity(pl3_rap, pl5_latin)[0][0])
print("Rap - rock: %.2f" % cosine_similarity(pl3_rap, pl6_rock)[0][0])
print("Rap - edm: %.2f" % cosine_similarity(pl3_rap, pl7_edm)[0][0])
print("Rap - klasika: %.2f" % cosine_similarity(pl3_rap, pl2_classic)[0][0])
print()
print("edm - pop: %.2f" % cosine_similarity(pl7_edm, pl8_pop)[0][0])
print("edm - rock: %.2f" % cosine_similarity(pl7_edm, pl6_rock)[0][0])
print("edm - latin: %.2f" % cosine_similarity(pl7_edm, pl5_latin)[0][0])

Podobnost playlistov:

Klasika - klasika: 1.00
Klasika - edm: 0.40

Rap - rap: 0.99
Rap - pop: 0.91
Rap - latin: 0.86
Rap - rock: 0.83
Rap - edm: 0.80
Rap - klasika: 0.39

edm - pop: 0.91
edm - rock: 0.88
edm - latin: 0.78


Opazimo, da je primerjava 2 playlistov klasike ali rapa 1, kar pomeni, da metoda primerjanja playlistov deluje. 

Vidimo da je rap playlista nabolj podobna pop playlisti, nato latino, rock, edm in na konca najmanj podobna kalsiki. 

Za edm pa vidimo, da je najbolj podobna popu in rocku. 

Z metodo podobnosti primerjamo se playliste in pesmi. S tem lahko v praksi neakj novo pojavljenih pesmi na platformi, primerjamo s playlisti razlicnih oseb, in pesem predlagamo osebam, ki so najbolj podobni.

In [93]:
song1_classic = make_vector_song(0, audio_df)
song1_classic

[0.0667,
 0.047,
 0.4236633273951368,
 0.0425,
 0.928,
 0.9,
 0.0652,
 0.0281,
 0.35406195591472023,
 0.4454987539150365,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0]

In [94]:
song1_classic = [make_vector_song(0, audio_df)]
song2_rap = [make_vector_song(5, audio_df)]
song3_edm = [make_vector_song(8, audio_df)]
song4_latin = [make_vector_song(17, audio_df)]
song5_rock = [make_vector_song(14, audio_df)]

In [95]:
print("Podobnost playlist in pesmi:\n")
print("Klasicna - klasicna: %.2f" % cosine_similarity(pl1_classic, song1_classic)[0][0])
print("Klasicna - edm: %.2f" % cosine_similarity(pl1_classic, song3_edm)[0][0])
print()
print("Rap - rap: %.2f" % cosine_similarity(pl3_rap, song2_rap)[0][0])
print("Rap - latin: %.2f" % cosine_similarity(pl3_rap, song4_latin)[0][0])
print("Rap - rock: %.2f" % cosine_similarity(pl3_rap, song5_rock)[0][0])
print("Rap - edm: %.2f" % cosine_similarity(pl3_rap, song3_edm)[0][0])
print("Rap - klasika: %.2f" % cosine_similarity(pl3_rap, song1_classic)[0][0])

Podobnost playlist in pesmi:

Klasicna - klasicna: 0.93
Klasicna - edm: 0.35

Rap - rap: 0.95
Rap - latin: 0.81
Rap - rock: 0.74
Rap - edm: 0.81
Rap - klasika: 0.32


Enako kot pri primerjanju playlistov opazimu tudi pri primerjnaju s pesmi. Ce primrjamo pesem klasike s playlistom klasike dobimo 93% podobnost, enako s pesmijo rapa.

Vidimo, da pri prierjavi pesmi klasike in playlista edm zelo mala podobnost in enako pri primerjavi rapa in klasike.

Ta metoda (primerjava vektorejv) za playliste in pesmi deluje dobro pri playlistih, kjer imamo pesmi istega zanra (vektor bo izracunan iz povprecja atributov in bo predstavljal to zvrst). Problem pa nastane, kjer imamo na playlistu polovico pesmi 1 zanra in polovico nasprotnega zanra (po vrednotih atributov). V takem primeru, bi se vrednosti atributov pokrajsali in bi dobili neke 3. vrednosti, ki niso podobni ne pesmim 1 zanra ne drugega.